In [63]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    import mysql.connector as mc 
    db = mc.connect(host=host,user=user,passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    data = (category,title,newsDate,newsTime,journalist,content,hit,url,source)
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(?,?,?,?,?,?,?,?,?)",data)
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    x = (commentDate,commentTime,commenter,commenterUrl,comment,url)
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(?,?,?,?,?,?)" ,x)
    db.commit()
def insert_newsRaw(db,category,url,newsRaw,source):
    cur = db.cursor()
    t = (category,url,newsRaw,source)
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(?,?,?,?)",t)
    db.commit()
def close(db):
    db.close()
#################連資料庫用



#################Exception Handler
def EH(breakpoint,newsDate=None,category=None,aLink=None):            
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + category + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: "+ breakpoint + "\n",
        "\t -Exception Traceback: \n", 
        str(traceback.format_exc()) + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler



#################get_all_pagelink
sentinel = object()
category=["政治","財經","國際"]
endday='20150505'
startday='20150501'
if endday is sentinel:
    endday=startday

def convert_day(d):
    return datetime.strptime(d, '%Y%m%d')

ed=convert_day(endday)
sd=convert_day(startday)

assert ed >= sd, "endday must after startday"

ds = []
for i in xrange(0,(ed-sd).days+1):
    ds.append(str((ed- timedelta(i)).strftime('%Y%m%d')))

url_all =[]
for newsDay in ds:
    url_dic = {"政治":"http://news.ltn.com.tw/newspaper/politics/%s"%newsDay,
                "財經":"http://news.ltn.com.tw/newspaper/business/%s"%newsDay,
                "國際":"http://news.ltn.com.tw/newspaper/world/%s"%newsDay,
                "社會":"http://news.ltn.com.tw/newspaper/society/%s"%newsDay,
                "焦點":"http://news.ltn.com.tw/newspaper/focus/%s"%newsDay,
                "地方":"http://news.ltn.com.tw/newspaper/local/%s"%newsDay,
                "娛樂":"http://news.ltn.com.tw/newspaper/entertainment/%s"%newsDay,
               }
    for cat in category:
        try:
            url_all.append(url_dic[str(cat)])
        except:
            print newsDay, cat, url_dic[str(cat)], "\n"
            print traceback.format_exc() + "\n"


aLink_all=[]
for cat in url_all:
    try:
        res = requests.get(cat)
        res.encoding="utf-8"
        soup = BeautifulSoup(res.text)
        p = soup.select('.tit')[0].text.encode('utf-8')
        #print type(p)
        b="".join(p.split("　"))
        c="".join(b.split(" "))
        page = re.search(".*共有(\d{1,4})筆.*",c).group(1)
        pageCount = int(math.ceil(float(page)/20))
        soups = [soup]
    except:
        print "error: ", cat
    if pageCount >1:
        for i in xrange(2,pageCount+1):
            curl = str(cat)+"?page=%s"%i
            res = requests.get(curl)
            res.encoding="utf-8"
            soups.append(BeautifulSoup(res.text))

    for soup in soups:         
        for block in soup.select(".picword"):
            try:
                aLink='http://news.ltn.com.tw'+block['href']
                aLink_all.append(aLink)
            except:
                print cat
                raise
    time.sleep(0)

for i in aLink_all:
    print i
    

error:  http://news.ltn.com.tw/newspaper/politics/20150505
error:  http://news.ltn.com.tw/newspaper/business/20150505
error:  http://news.ltn.com.tw/newspaper/world/20150505
error:  http://news.ltn.com.tw/newspaper/politics/20150504
error:  http://news.ltn.com.tw/newspaper/business/20150504
error:  http://news.ltn.com.tw/newspaper/world/20150504
error:  http://news.ltn.com.tw/newspaper/politics/20150503
error:  http://news.ltn.com.tw/newspaper/business/20150503
error:  http://news.ltn.com.tw/newspaper/world/20150503
error:  http://news.ltn.com.tw/newspaper/politics/20150502
error:  http://news.ltn.com.tw/newspaper/business/20150502
error:  http://news.ltn.com.tw/newspaper/world/20150502
error:  http://news.ltn.com.tw/newspaper/politics/20150501
error:  http://news.ltn.com.tw/newspaper/business/20150501
error:  http://news.ltn.com.tw/newspaper/world/20150501
http://news.ltn.com.tw/news/world/paper/876427
http://news.ltn.com.tw/news/world/paper/876428
http://news.ltn.com.tw/news/world/pa

In [54]:
res = requests.get(cat)
res.encoding="utf-8"
soup = BeautifulSoup(res.text)
p = soup.select('.tit')[0]

print p.text
page = re.search(".*共有(\d{1,4})筆.*",c).group(1)
print page

2015-05-05　共有 11 筆政治新聞
11


In [45]:
a="2015-05-05　共有 11 筆政治新聞"
b="".join(a.split("　"))
c="".join(b.split(" "))
print c

2015-05-05共有11筆政治新聞


In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(%s,%s,%s,%s)",[category,url,newsRaw,source])
    db.commit()
def get_newsRaw(db,url):
    cur = db.cursor()
    cur.execute("select newsRaw from newsRaw where url = %s",[url])
    return cur.fetchone()
def close(db):
    db.close()
#################連資料庫用


#################Exception Handler
def EH(breakpoint,newsDate=None,category=None,aLink=None):            
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + category + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: "+ breakpoint + "\n",
        "\t -Exception Traceback: \n", 
        str(traceback.format_exc()) + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler



#################get_all_pagelink
sentinel = object()
def get_all_pagelink(startday=(str(date.today()-timedelta(1))),endday=sentinel,category=["政治","財經","國際","焦點"],sleep_time=0):
    
    if endday is sentinel:
        endday=startday
    
    def convert_day(d):
        return datetime.strptime(d, '%Y%m%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)

    assert ed >= sd, "endday must after startday"
    
    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y%m%d')))

    url_all =[]
    for newsDay in ds:
        url_dic = {"政治":"http://news.ltn.com.tw/newspaper/politics/%s"%newsDay,
                    "財經":"http://news.ltn.com.tw/newspaper/business/%s"%newsDay,
                    "國際":"http://news.ltn.com.tw/newspaper/world/%s"%newsDay,
                    "社會":"http://news.ltn.com.tw/newspaper/society/%s"%newsDay,
                    "焦點":"http://news.ltn.com.tw/newspaper/focus/%s"%newsDay,
                    "地方":"http://news.ltn.com.tw/newspaper/local/%s"%newsDay,
                    "娛樂":"http://news.ltn.com.tw/newspaper/entertainment/%s"%newsDay,
                   }
        for cat in category:
            try:
                url_all.append(url_dic[str(cat)])
            except:
                print newsDay, cat, url_dic[str(cat)], "\n"
                print traceback.format_exc() + "\n"
    
    aLink_all=[]
    for cat in url_all:
        try:
            res = requests.get(cat)
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            p = soup.select('.tit')[0].text.encode('utf-8')
            print type(p)
            b="".join(p.split("　"))
            c="".join(b.split(" "))
            page = re.search(".*共有(\d{1,4})筆.*",c).group(1)
            pageCount = int(math.ceil(float(page)/20))
            soups = [soup]
        except:
            print cat+ "\n"
            print traceback.format_exc()+ "\n"
        if pageCount >1:
            for i in xrange(2,pageCount+1):
                curl = str(cat).split(".htm")[0] + "-" + str(i) + ".htm"
                res = requests.get(curl)
                res.encoding="utf-8"
                soups.append(BeautifulSoup(res.text))

        for soup in soups:         
            for block in soup.select(".lipic > a"):
                try:
                    aLink=block.select('a')[0]['href']
                    aLink_all.append(aLink)
                except:
                    print cat
                    raise
        time.sleep(sleep_time)
    
    return aLink_all    
#################get_all_pagelink



#################crawl_all_rawpage_and_insert_into_db
def all_rawpage(aLink_all,db="practice",passwd="oblivn0374",port="localhost",user="root"):
    
        #driver = webdriver.Firefox()
        #driver = webdriver.Chrome()
        driver = webdriver.PhantomJS()
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
        webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
    
    for aLink in aLink_all:
        #########rawpage breakpoint
        try:
            driver.get(aLink)
            WebDriverWait(driver, 30).until(
                    EC.visibility_of_element_located((By.ID, "fbComments"))
                )
            html = driver.page_source
            cont = str(html.encode('utf-8'))
            conn=connect(db,passwd,port,user)
            insert_newsRaw(conn,aLink,cont)
        except Exception as e:
            EH('rawpage',newsDate=None,category=None,aLink=None,e=e) 
            print "rawpage not write in DB \n", traceback.format_exc(),aLink,"\n"
            continue
        finally:
            driver.quit()
            close(conn)
        #########rawpage breakpoint                
#################crawl_all_rawpage_and_insert_into_db



#################parse_page_and_insert_into_db
identify = string.maketrans('', '')
delEStr = string.punctuation
hit = None
sentinel = object()
def parse_page(aLink_all,db="practice",passwd="oblivn0374",port="localhost",user="root"):
     for aLink in aLink_all:
        #########retrieve_rawpage_from_db
        try:
            conn=connect(db,passwd,port,user)
            cont=get_newsRaw(conn,aLink)
        except:
            print "retrieve url not correct", traceback.format_exc(),aLink,"\n"
        #########retrieve_rawpage_from_db
        
        #########category breakpoint
        $#category=block.select('em')[0].text
        #########category breakpoint
        
        #########title breakpoint
        $#title=block.select('a')[0].text
        #########title breakpoint
        
        ########Date&Time breakpoint
        $#newsDate=newsReg.group(1)+newsReg.group(2)
        $#newsTime=timeRegex.group(3)
        ########Date&Time breakpoint

        #########content breakpoint
        try:
            soupCont = BeautifulSoup(cont)
            content=""
            contentRaw = soupCont.select('.story > sectione > p')
            try:
                for i in xrange(0,len(contentRaw)):
                    for con in contentRaw[i].select('img'):
                        contentRaw[i].img.decompose()
            except:
                pass
            for p in contentRaw:
                content = content + p.text
            content=content.strip()
        except Exception as e:
            EH('content',newsDate=None,category=None,aLink=None,e=e) 
        #########content breakpoint        
        
        #########journist breakpoint
        jor = str(soupCont.select('.story')[0].prettify().encode("utf-8"))
        try:
            journalist=re.search('(([^圖].*?)／.*?(?:報導|特稿|專訪).*?|本報訊)', jor).group(1).strip()
            journalist=journalist.translate(identify, delEStr)
            if '記者' in journalist:
                journalist=re.sub('記者','',journalist)
            journalist=re.sub('、', '', journalist)
        except:
            try:
                journalist=re.search('文[／．](.*?)<',cont).group(1).strip()
                journalist=journalist.translate(identify, delEStr)
                journalist=re.sub('、', '', journalist)
            except Exception as e:
                try:
                    journalist=None
                    EH('journalist',newsDate=None,category=None,aLink=None,e=e) 
                except:
                    print "jor: ",aLink," WTF"
        #########journist breakpoint

        #########comment breakpoint
        comments=[]
        try:
            commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
            commLink1= commLink0.select(' span > iframe')[0]['src']
            commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
            resComm = requests.get(commLink)
            resComm.encoding = "utf-8"
            soupComm = BeautifulSoup(resComm.text)      
            commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
            for comm in commentBlock: 
                dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                commentTime=dt.group(4)
                commenter=comm.select('.profileName')[0].text
                commenterUrl=comm.select('.profileName')[0]['href']
                comment=comm.select('.postText')[0].text
                comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
        except Exception as e:
            EH('comment',newsDate=None,category=None,aLink=None,e=e) 
        #########comment breakpoint

        #################進資料庫用
        #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
        try:
            insert_news(conn,$category,$title,$newsDate,$newsTime,journalist,content,hit,aLink)
            for c in comments:
                insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
        except:
            print "something not write in DB \n", traceback.format_exc(),aLink,"\n"
        finally:
            close(conn)
        #################進資料庫用
#################parse_page_and_insert_into_db    